In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")

In [4]:
df.head(20)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [5]:
df1 = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [6]:
df["text"] = df["text"].apply(lambda x: x.lower())
df1["text"] = df1["text"].apply(lambda x: x.lower())

In [7]:
import re

In [8]:
def remove_URL(text):
    return re.sub(r"https?://\S+|www\.\S+", "", text)

df["text"] = df["text"].apply(lambda x: remove_URL(x))
df1["text"] = df1["text"].apply(lambda x: remove_URL(x))

In [9]:
def remove_html(text):
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)

df["text"] = df["text"].apply(lambda x: remove_html(x))
df1["text"] = df1["text"].apply(lambda x: remove_html(x))

In [10]:
import string

In [11]:
def remove_punct(text):
    return text.translate(str.maketrans('', '', string.punctuation))


df["text"] = df["text"].apply(lambda x: remove_punct(x))
df1["text"] = df1["text"].apply(lambda x: remove_punct(x))

In [12]:
df = df.drop('keyword', axis=1)
df1 = df1.drop('keyword', axis=1)

In [13]:
df = df.drop('location', axis=1)
df1 = df1.drop('location', axis=1)

In [14]:
import nltk
from nltk import TweetTokenizer

tokenizer = TweetTokenizer()

df['tokens'] = [tokenizer.tokenize(item) for item in df.text]
df1['tokens'] = [tokenizer.tokenize(item) for item in df1.text]

In [15]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_item(item):
    new_item = []
    for x in item:
        x = lemmatizer.lemmatize(x)
        new_item.append(x)
    return " ".join(new_item)

df['tokens'] = [lemmatize_item(item) for item in df.tokens]
df1['tokens'] = [lemmatize_item(item) for item in df1.tokens]

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()


X = vectorizer.fit_transform(df.text).toarray()
y = df['target']

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [18]:
from sklearn.naive_bayes import MultinomialNB

In [19]:
mb = MultinomialNB()
mb.fit(X_train,y_train)

MultinomialNB()

In [20]:
y_pred = mb.predict(X_test)

In [21]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [22]:
acc = accuracy_score(y_test,y_pred)
print("The accuracy of the model is :",acc)

The accuracy of the model is : 0.8025394045534151


In [23]:
cf = confusion_matrix(y_test,y_pred)
cf

array([[1157,  181],
       [ 270,  676]])

In [24]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.86      0.84      1338
           1       0.79      0.71      0.75       946

    accuracy                           0.80      2284
   macro avg       0.80      0.79      0.79      2284
weighted avg       0.80      0.80      0.80      2284



In [25]:
test = vectorizer.transform(df1.text).toarray()

In [26]:
pred_1 = mb.predict(test)

In [27]:
sub = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [28]:
sub.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [29]:
sub['target'] = pred_1
sub.to_csv('submission.csv',index=False)

In [30]:
r =pd.read_csv("submission.csv")
r

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1
